In [3]:
from __future__ import division

import numpy as np
from gensim.models import Word2Vec 
import scipy.stats as stat
from scipy.special import gamma
import scipy
from numpy import pi
import time
import line_profiler
import cProfile
import IPython
ip = IPython.get_ipython()
ip.define_magic('lprun', line_profiler.magic_lprun)
%load_ext line_profiler
%load_ext autoreload


In [6]:
wordvecs = Word2Vec.load_word2vec_format(
    "/Users/michael/Documents/Gaussian_LDA-master/data/glove.wiki/glove.6B.50d.txt", binary=False)

In [19]:
import FastGaussianLDA
%aimport FastGaussianLDA
%autoreload 1
# import GaussianLDA
# reload(Gauss_LDA)
# reload(GaussianLDA)
start  = time.time()

corpus = ['apple banana mango melon',
 'barrel cannon gun war',
 'democrat republican senate politics']
corpus2 = ["apple orange mango melon ", "canvas art mural paint painting ", "pineapple kiwi grape strawberry ",
              "picture frame picasso sculpture art ", "coconut guava blueberry blackberry ", "statue monument art artist "]
corpus2 = [sent * 5 for sent in corpus2]*4
wordvec_fileapth = "/Users/michael/Documents/Gaussian_LDA-master/data/glove.wiki/glove.6B.50d.txt"
g = FastGaussianLDA.Gauss_LDA(3, corpus2, word_vector_model=wordvecs)
g.fit(100)
print (time.time()-start)

Done processing corpus with 24 documents
There are 24 words that could be convereted to word vectors in your corpus 
There are 0 words that could NOT be converted to word vectors
Intialization complete
Starting fit
0 iterations complete
1 iterations complete
2 iterations complete
3 iterations complete
4 iterations complete
5 iterations complete
6 iterations complete
7 iterations complete
8 iterations complete
9 iterations complete
topic assignments are whack
topic assignments are whack
topic assignments are whack
topic assignments are whack
topic assignments are whack
topic assignments are whack
topic assignments are whack
topic assignments are whack
10 iterations complete
11 iterations complete
12 iterations complete
13 iterations complete
14 iterations complete
15 iterations complete
16 iterations complete
17 iterations complete
18 iterations complete
19 iterations complete
20 iterations complete
21 iterations complete
22 iterations complete
23 iterations complete
24 iterations compl

In [10]:
g.word_vecs

{}

old time was 237 secs on 30 iterations without NUMBA

In [20]:
mean1 = g.topic_params[0]["Topic Mean"]; mean2 = g.topic_params[1]["Topic Mean"]; mean3 = g.topic_params[2]["Topic Mean"]
wordvecs.most_similar(positive=[mean1])

[(u'sculpture', 0.8271721601486206),
 (u'painting', 0.8245132565498352),
 (u'art', 0.803730845451355),
 (u'collection', 0.7746897339820862),
 (u'garden', 0.7652238011360168),
 (u'famous', 0.7629948854446411),
 (u'works', 0.7435003519058228),
 (u'paintings', 0.7393991947174072),
 (u'sculptures', 0.7150182723999023),
 (u'glass', 0.7109430432319641)]

In [21]:
wordvecs.most_similar(positive=[mean2])

[(u'colored', 0.8344143629074097),
 (u'painted', 0.8143922090530396),
 (u'canvas', 0.8039060831069946),
 (u'paint', 0.8033887147903442),
 (u'green', 0.8015591502189636),
 (u'pink', 0.781296968460083),
 (u'shaped', 0.7649050951004028),
 (u'shiny', 0.7583204507827759),
 (u'piece', 0.7541706562042236),
 (u'bright', 0.7407790422439575)]

In [22]:
wordvecs.most_similar(positive=[mean3])

[(u'mango', 0.8706889152526855),
 (u'pineapple', 0.8398306369781494),
 (u'coconut', 0.8316406011581421),
 (u'peach', 0.8135504722595215),
 (u'fruit', 0.8121268153190613),
 (u'apricot', 0.7849801778793335),
 (u'juice', 0.7718098163604736),
 (u'plum', 0.7694075107574463),
 (u'banana', 0.7686874866485596),
 (u'cherry', 0.7675427198410034)]

In [ ]:
newdoc = "fruit apple orange mango taco painting sculpture book art artist picasso"

In [ ]:
def myfunc(mat):
    np.linalg.det(mat)
    return np.linalg.inv(mat)
    
%prun myfunc(np.arange(100).reshape(10,10))

In [6]:
a = None
if a: print 'yes'